![banner](./images/banner.png "banner")

# <font color=#6290C3>Modelo de aprendizaje automático para la predicción del Ratio Internacional Normalizado (INR) en pacientes bajo terapia con Antagonistas de la Vitamina K </font>

1. [Descripción del proyecto](#project-description)  
    1.1 [Objetivos](#project-description-goal)  
    1.2 [Data](#project-description-data)  
    1.3 [Software](#project-description-software)  

2. [Exploración y visualización de datos](#project-description)  
    2.1 [Carga de datos](#project-description-goal)  
    2.2 [Generación del csv](#project-description-data)  


<h2 id="project-description"><font color="#6290C3">1. Descripción del proyecto</font></h2>
Este proyecto consiste en una predición de datos ....

- Es una tarea supervisada, es decir, el modelo se ha de entrar con muchos datos. Una vez entrenado el modelo, lo aplicamos a una nueva fila donde falta un valor, aquello que queremos predecir, y si el entrenamiento fue bueno, el modelo podrá predecir ese dato que falta. Las columnas para las que sí conocemos siempre todos los valores se llaman "características" o "features" en inglés.

- El dato que les falta a esas nuevas filas es el correspondiente a la columna 'target' (también llamado "destino" "objetivo" o "outcome") y es lo que intentamos predecir, en este caso, es la clase a la que pertenece cada fila.

- La columna 'target' tiene valores continuos, por lo que usaremos modelos de regresión.

- Los datos de nuestra columna "target" puede tomar distintos valores entre los rangos 0.1 y 3.0

- Si los datos no hubieran sido etiquetados (es decir, las clases no estuvieran definidas desde el inicio), necesitaríamos un modelo de aprendizaje automático no supervisado (es decir, el modelo debe encontrar los grupos o 'clusters' en inglés).

Ajustaremos los modelos de regresión de aprendizaje automático más populares a unos datos de pacientes con anticoagulantes y seleccionaremos el modelo con mejor rendimiento. Para verificar el rendimiento del modelo, separamos aleatoriamente el conjunto de datos en 3 subconjuntos:

- Un subconjunto de datos para entrenar el modelo: representan los datos 'antiguos', donde no faltan valores, es decir, todos los valores de todas las columnas incluida la 'target' son conocidos, así sabemos si eses pacientes sufrieron o no fallo cardíaco,

- Un subconjunto de los datos para validar el modelo: representan datos 'nuevos' porque reservamos los valores de la columna 'target', los separamos y los guardamos, es decir, pretendemos que no existen, como si fueran nueves pacientes, le damos ese subconjunto de validación al modelo ya entrenado y comparamos los resultados del modelo, es decir, sus prediciones, con los valores reales que teníamos reservados. Como ajustaremos los parámetros para hacer que los resultados del modelo se acerquen cada vez más a los valores reales, el modelo verá este conjunto de datos de validación con mucha frecuencia, lo que puede hacer que el modelo se desvíe, muestre preferencia o sesgo ('bias' en inglés) hacia el conjunto de datos de validación (explicamos la definición de sesgo en los siguientes Jupyter Notebooks sobre Explicabilidad XAI.ipynb y Cuantificación de Incertidumbre UQ.ipynb en este mismo repositorio), es decir, puede estar sobreajustado ('overfitted') y no generalizar bien cuando lo aplicamos a otros datos, por eso separamos un último grupo:

- Un subconjunto de los datos para probar los resultados del modelo: para la verificación final que compara los resultados del modelo ajustado con datos que el modelo entrenado y validado nunca ha visto antes; es la mejor manera que tenemos de simular nuevos datos reales. 

<h3 id="project-description-goal" style="color:#6290C3;">1.1 Objetivos</h3>

El objetivo es predecir el INR de un paciente con anticoagulantes mediante varaibles analiticas y ritmo de vida... y otras características consideradas factores de riesgo. Las conclusiones de este análisis pueden ayudar en la detección temprana y la prevención de diagnosticos complicados.

<h3 id="project-description-goal" style="color:#6290C3;">1.2 Datos</h3>

El conjunto de datos que usamos para entrenar nuestro modelo son datos sinteticos de Synthnea, un software opensource de estados unidos. Este programa generará mediante sus comandos historias completas de pacientes.

Para ejecutar el programa ejecutremos `java -jar synthea.jar` este procesará los valores que se indiquen en el archivo de configuración **synthea.properties** indicaremos lo siguiente:

- default_population = 5.000 -> cantida de pacientes
- exporter.csv.export = true  -> se exportaran los datos a ficheros csv
- exporter.csv.append_mode = true   -> si se ejecuta otra vez el archivo se actualizaran los datos del csv
- generate.modules = cardiovascular_disease, atrial_fibrillation, venous_thromboembolism  -> permite priorizar la generación de pacientes con estas patologias.


Synthea export data as CSV into `./output/csv`.  Las historias de pacientes se reparten en los siguientes csv.

| File | Description |
|------|-------------|
| [`allergies.csv`](#allergies) | Patient allergy data. |
| [`careplans.csv`](#careplans) | Patient care plan data, including goals. |
| [`claims.csv`](#claims) | Patient claim data. |
| [`claims_transactions.csv`](#claims-transactions) | Transactions per line item per claim. |
| [`conditions.csv`](#conditions) | Patient conditions or diagnoses. |
| [`devices.csv`](#devices) | Patient-affixed permanent and semi-permanent devices. |
| [`encounters.csv`](#encounters) | Patient encounter data. |
| [`imaging_studies.csv`](#imaging-studies) | Patient imaging metadata. |
| [`immunizations.csv`](#immunizations) | Patient immunization data. |
| [`medications.csv`](#medications) | Patient medication data. |
| [`observations.csv`](#observations) | Patient observations including vital signs and lab reports. |
| [`organizations.csv`](#organizations) | Provider organizations including hospitals. |
| [`patients.csv`](#patients) | Patient demographic data. |
| [`payer_transitions.csv`](#payer-transitions) | Payer Transition data (i.e. changes in health insurance). |
| [`payers.csv`](#payers) | Payer organization data. |
| [`procedures.csv`](#procedures) | Patient procedure data including surgeries. |
| [`providers.csv`](#providers) | Clinicians that provide patient care. |
| [`supplies.csv`](#supplies) | Supplies used in the provision of care. |


Para nuestro analisis solo utilizaremos:

# Observations
| | Column Name | Data Type | Required? | Description |
|-|-------------|-----------|-----------|-------------|
| | Date | iso8601 UTC Date (`yyyy-MM-dd'T'HH:mm'Z'`) | `true` | The date and time the observation was performed. |
| | Patient | UUID | `true` | Foreign key to the Patient. |
| | Encounter | UUID | `true` | Foreign key to the Encounter where the observation was performed. |
| | Category | String | `false` | Observation category. |
| | Code | String | `true` | Observation or Lab code from LOINC |
| | Description | String | `true` | Description of the observation or lab. |
| | Value | String | `true` | The recorded value of the observation. Often numeric, but some values can be verbose, for example, multiple-choice questionnaire responses. |
| | Units | String | `false` | The units of measure for the value. |
| | Type | String | `true` | The datatype of `Value`: `text` or `numeric` |


# Patients
| | Column Name | Data Type | Required? | Description |
|-|-------------|-----------|-----------|-------------|
| | Id | UUID | `true` | Primary Key. Unique Identifier of the patient. |
| | BirthDate | Date (`YYYY-MM-DD`) | `true` | The date the patient was born. |
| | DeathDate | Date (`YYYY-MM-DD`) | `false` | The date the patient died. |
| | SSN | String | `true` | Patient Social Security identifier. |
| | Drivers | String | `false` | Patient Drivers License identifier. |
| | Passport | String | `false` | Patient Passport identifier. |
| | Prefix | String | `false` | Name prefix, such as `Mr.`, `Mrs.`, `Dr.`, etc. |
| | First | String | `true` | First name of the patient. |
| | Middle | String | `false` | Middle name of the patient. |
| | Last | String | `true` | Last or surname of the patient. |
| | Suffix | String | `false` | Name suffix, such as `PhD`, `MD`, `JD`, etc. |
| | Maiden | String | `false` | Maiden name of the patient. |
| | Marital | String | `false` | Marital Status. `M` is married, `S` is single. Currently no support for divorce (`D`) or widowing (`W`) |
| | Race | String | `true` | Description of the patient's primary race. |
| | Ethnicity | String | `true` | Description of the patient's primary ethnicity. |
| | Gender | String | `true` | Gender. `M` is male, `F` is female. |
| | BirthPlace | String | `true` | Name of the town where the patient was born. |
| | Address | String | `true` | Patient's street address without commas or newlines. |
| | City | String | `true` | Patient's address city. |
| | State | String | `true` | Patient's address state. |
| | County | String | `false` | Patient's address county. |
| | FIPS County Code | String | `false` | Patient's FIPS county code. |
| | Zip | String | `false` | Patient's zip code. |
| | Lat | Numeric | `false` | Latitude of Patient's address. |
| | Lon | Numeric | `false` | Longitude of Patient's address. |
| | Healthcare_Expenses | Numeric | `true` | The total lifetime cost of healthcare to the patient (i.e. what the patient paid). |
| | Healthcare_Coverage | Numeric | `true` | The total lifetime cost of healthcare services that were covered by Payers (i.e. what the insurance company paid). |
| | Income | Numeric | `true` | Annual income for the Patient |


El variable 'target', es decir, lo que queremos predecir, se llama "INR", y es una variable continua que toma las siguientes interpretaciones:

- <2.0: sí, es decir, pacientes que sí presentan riesgo de padecer una cardiopatía,
- 2.0 y 3.0: no, es decir, pacientes que no presentan riesgo de padecer una cardiopatía.
- 3.0: no, es decir, pacientes que no presentan riesgo de padecer una cardiopatía.

## 1.3 Software

Importamos las siguientes librerías:

In [2]:
import pandas as pd
import numpy as np

# data exploration and preparation  
#from sklearn.metrics import mutual_info_score, roc_auc_score
#from sklearn.model_selection import train_test_split
#from sklearn.feature_extraction import DictVectorizer 
#from sklearn.preprocessing import StandardScaler 

# machine learning models
#from sklearn.linear_model import LogisticRegression
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import RandomForestClassifier


# model evaluation
#from sklearn.metrics import accuracy_score, f1_score, auc, recall_score, precision_score, confusion_matrix
#from sklearn.metrics import make_scorer
#from sklearn.model_selection import GridSearchCV, KFold, cross_val_score


# plotting and displaying in the notebook
#import seaborn as sns
#from matplotlib import pyplot as plt
#from IPython.display import display
#from sklearn import tree


# ignore warnings
#import warnings
#warnings.filterwarnings("ignore")

#%matplotlib inline

# 2. Exploración y visualización de datos

## 2.1 Carga de datos

Vamos a cargar los datos del fichero `patients.csv`:

In [3]:
df_patients = pd.read_csv("data/patients.csv")
df_patients.head()
#df_patients.info()

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,MIDDLE,LAST,...,CITY,STATE,COUNTY,FIPS,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,INCOME
0,b7822130-1517-3e23-decc-f2ed09c895e6,1968-04-30,NaN,999-69-8092,S99943430,X29170225X,Mrs.,Kiersten731,Julianna856,O'Hara248,...,Maynard,Massachusetts,Middlesex County,25017.0,1754,42.413757,-71.432504,43696.32,1163198.71,103064
1,8a61c160-ef39-fc94-5d34-058856eb980f,1967-03-07,NaN,999-94-9323,S99975833,X54176475X,Ms.,Elwanda490,Angele108,Smith67,...,Woburn,Massachusetts,Middlesex County,25017.0,1890,42.468290,-71.099257,297576.83,1671648.53,29712
2,417c2332-884f-5d92-7aaa-34fdc4210fc0,1960-04-26,NaN,999-54-9267,S99991952,X68442483X,Mrs.,Sandi885,NaN,Leannon79,...,East Longmeadow,Massachusetts,Hampden County,NaN,0,42.109412,-72.463043,386536.66,587114.90,179106
3,5acd3be8-56b2-6fb0-10ae-7cce6695f456,1971-06-19,NaN,999-79-9186,S99982125,X60103449X,Mr.,Giuseppe872,Mike230,Douglas31,...,Hudson,Massachusetts,Middlesex County,25017.0,1749,42.410253,-71.548368,148803.35,5390.13,87760
4,e5c7b433-e52b-b22f-1eb3-b6bbdc03cdef,1970-01-16,NaN,999-88-4683,S99957545,X7028318X,Mrs.,Fairy757,Candie120,Gutmann970,...,Lee,Massachusetts,Berkshire County,25003.0,1238,42.338015,-73.194752,737551.25,540698.89,64021


El conjunto de datos contiene información de **5,826 pacientes y 28 variables que describen sus características básicas y demográficas**, como edad, género, etnia, estado civil, lugar de nacimiento e ingresos. Incluye además datos administrativos (por ejemplo, número de seguro social o pasaporte) y médicos (como cobertura de salud). En general, ofrece una visión completa del perfil socio-demográfico de cada paciente.

Vamos a cargar los datos del fichero `observations.csv`:

In [4]:
df_observations = pd.read_csv("data/observations.csv")
df_observations.head()
#df_observations.info()

,DATE,PATIENT,ENCOUNTER,CATEGORY,CODE,DESCRIPTION,VALUE,UNITS,TYPE
0,2016-09-20T12:48:33Z,b7822130-1517-3e23-decc-f2ed09c895e6,b7822130-1517-3e23-2b0f-dfe3a7340133,laboratory,4548-4,Hemoglobin A1c/Hemoglobin.total in Blood,6.2,%,numeric
1,2016-09-20T12:48:33Z,b7822130-1517-3e23-decc-f2ed09c895e6,b7822130-1517-3e23-2b0f-dfe3a7340133,vital-signs,8302-2,Body Height,162.5,cm,numeric
2,2016-09-20T12:48:33Z,b7822130-1517-3e23-decc-f2ed09c895e6,b7822130-1517-3e23-2b0f-dfe3a7340133,vital-signs,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,2.0,{score},numeric
3,2016-09-20T12:48:33Z,b7822130-1517-3e23-decc-f2ed09c895e6,b7822130-1517-3e23-2b0f-dfe3a7340133,vital-signs,29463-7,Body Weight,80.9,kg,numeric
4,2016-09-20T12:48:33Z,b7822130-1517-3e23-decc-f2ed09c895e6,b7822130-1517-3e23-2b0f-dfe3a7340133,vital-signs,39156-5,Body mass index (BMI) [Ratio],30.6,kg/m2,numeric


Este conjunto de datos es considerablemente amplio, con **4,627,376 registros y 9 variables** que describen distintas interacciones clínicas de los pacientes. Contiene información sobre la fecha, tipo de encuentro, categoría médica, código y descripción de cada evento, así como los valores y unidades asociados. Representa un registro detallado de las actividades y mediciones realizadas en el contexto sanitario.

Dentro de este conjunto se identifica nuestra **variable objetivo 'target', que indica si el paciente cuenta con un registro de INR** y el valor correspondiente. Esta variable permite evaluar la presencia y magnitud de dicha medición, siendo la clave de este análisis.

## 2.2 Generación del csv

Se filtran los registros para incluir únicamente **pacientes vivos** y aquellos con **controles de INR**, dado que el análisis se enfoca en esta medición. A continuación, se procede a **limpiar y depurar** los datos del subconjunto obtenido, asegurando su calidad y consistencia para el estudio.

**Pacientes difuntos**  
La columna DeathDate indica la fecha de fallecimiento; se eliminarán todos los registros donde este valor no sea nulo, conservando únicamente a los pacientes vivos para el análisis.

In [5]:
print("Cantidad de pacientes vivos:")
print(df_patients['DEATHDATE'].isna().sum())

print("Cantidad de pacientes difuntos:")
print(df_patients['DEATHDATE'].notna().sum())


Cantidad de pacientes vivos:
5000
Cantidad de pacientes difuntos:
826


Actualmente contamos con **5,000 pacientes vivos y 826 pacientes difuntos**. Para el análisis, conservaremos únicamente a los pacientes vivos, eliminando los registros correspondientes a los fallecidos.

In [6]:
df_patients = df_patients[df_patients['DEATHDATE'].isna()]

Una vez filtrados, de cada paciente solo nos interesan tres variables clave: edad, género e identificador, que serán utilizadas para los análisis posteriores.

Para la variable **edad**, primero calculamos los años a partir de la fecha de nacimiento utilizando la diferencia con la fecha actual. Aquellos pacientes que no cuenten con fecha de nacimiento registrada tendrán su edad rellenada con la mediana del conjunto, asegurando que no queden valores nulos y manteniendo la coherencia del dataset.

In [7]:
#convertimos la fecha de nacimiento en formato datetime para el calculo
df_patients['BIRTHDATE'] = pd.to_datetime(df_patients['BIRTHDATE'], errors='coerce')

df_patients['AGE'] = ((pd.Timestamp('today') - df_patients['BIRTHDATE']).dt.days / 365.25).astype(int)
df_patients['AGE'] = df_patients['AGE'].fillna(df_patients['AGE'].median())
df_patients = df_patients[['Id','AGE', 'GENDER']]

**Pacientes con INR**  
A continuación, identificaremos a los pacientes que toman anticoagulantes y sus mediciones asociadas, filtrando únicamente las observaciones cuya descripción incluya INR. Esto nos permite centrar el análisis en los registros relevantes.

In [8]:
num_inr = (df_observations['DESCRIPTION'].str.contains('INR', case=False, na=False)).sum()
print("Cantidad de observaciones que contienen INR en la descripción:", num_inr)

num_not_inr = (~df_observations['DESCRIPTION'].str.contains('INR', case=False, na=False)).sum()
print("Cantidad de observaciones que NO contienen INR en la descripción:", num_not_inr)


Cantidad de observaciones que contienen INR en la descripción: 2083
Cantidad de observaciones que NO contienen INR en la descripción: 4625293


Contamos con **2,083 observaciones correspondientes a pacientes con medición de INR**. Para el análisis, conservaremos únicamente estos pacientes que poseen dicha observación.

In [9]:
# 1️⃣ Obtener los IDs de los pacientes que tienen 'INR' en alguna descripción
patient_ids_inr = df_observations.loc[
    df_observations['DESCRIPTION'].str.contains('INR', case=False, na=False),
    'PATIENT'  # reemplaza con el nombre de la columna de IDs de pacientes
].unique()

# 2️⃣ Filtrar todo el dataset para dejar solo observaciones de esos pacientes
df_observations_filtered = df_observations[df_observations['PATIENT'].isin(patient_ids_inr)]

# 3️⃣ Opcional: ver cuántas filas quedan
print("Cantidad de observaciones de pacientes con al menos una descripción con INR:", len(df_observations_filtered))



Cantidad de observaciones de pacientes con al menos una descripción con INR: 692282


Una vez filtrados, de cada paciente solo nos interesan tres variables clave: paciente, descripción y valores (incluimos las unidades).

In [10]:
df_observations = df_observations[['PATIENT','DESCRIPTION', 'VALUE', 'UNITS']]
df_observations.head(5)

,PATIENT,DESCRIPTION,VALUE,UNITS
0,b7822130-1517-3e23-decc-f2ed09c895e6,Hemoglobin A1c/Hemoglobin.total in Blood,6.2,%
1,b7822130-1517-3e23-decc-f2ed09c895e6,Body Height,162.5,cm
2,b7822130-1517-3e23-decc-f2ed09c895e6,Pain severity - 0-10 verbal numeric rating [Sc...,2.0,{score}
3,b7822130-1517-3e23-decc-f2ed09c895e6,Body Weight,80.9,kg
4,b7822130-1517-3e23-decc-f2ed09c895e6,Body mass index (BMI) [Ratio],30.6,kg/m2


A continuación, uniremos la información de los pacientes con sus respectivas observaciones utilizando el identificador único (Id) de cada paciente. Esta combinación nos permitirá consolidar los datos en un solo dataset, facilitando el análisis y asegurando que toda la información relevante de cada paciente esté centralizada.

In [11]:
# - df_patients (izquierda) tiene la columna clave 'Id'
# - df_observations (derecha) tiene la columna clave 'PATIENT'

df_patients_observations = pd.merge(
    df_patients,        # DataFrame de la izquierda
    df_observations,  # DataFrame de la derecha
    left_on='Id',           # <--- Columna clave en el DataFrame de la izquierda (df_patients)
    right_on='PATIENT',     # <--- Columna clave en el DataFrame de la derecha (df_observations)
    how='left'              # Tipo de unión (mantiene todos los pacientes del df_patients)
)

#Eliminamos la columna PATIENT para no tener duplicados en el identificador
columnas_a_eliminar = ['PATIENT']
df_patients_observations = df_patients_observations.drop(columns=columnas_a_eliminar)
df_patients_observations.to_csv('data/patients_observations.csv')
df_patients_observations.head()

,Id,AGE,GENDER,DESCRIPTION,VALUE,UNITS
0,b7822130-1517-3e23-decc-f2ed09c895e6,57,F,Hemoglobin A1c/Hemoglobin.total in Blood,6.2,%
1,b7822130-1517-3e23-decc-f2ed09c895e6,57,F,Body Height,162.5,cm
2,b7822130-1517-3e23-decc-f2ed09c895e6,57,F,Pain severity - 0-10 verbal numeric rating [Sc...,2.0,{score}
3,b7822130-1517-3e23-decc-f2ed09c895e6,57,F,Body Weight,80.9,kg
4,b7822130-1517-3e23-decc-f2ed09c895e6,57,F,Body mass index (BMI) [Ratio],30.6,kg/m2


### Simplificación de las variables y pivotación

.py para los nombres de las variables y pivotación el csv de observaciones

In [1]:
df_patients_observations['DESCRIPTION'].unique()

NameError: name 'df_patients_observations' is not defined

In [79]:
# Reload the module to get the new functions
import importlib
from utils.cleaner_category_match import *


# First, diagnose what's happening
#diagnose_category_match(df_patients_observations, 'urinalysis','microbiology','infectious','immunology_allergy','oncology','ophthalmology_imaging','mental_health','social_determinants','demographics','substance_use','administrative','other', column='DESCRIPTION')
diagnose_category_match(df_patients_observations, 'urinalysis', column='DESCRIPTION')

# Then try dropping
#df_observations_clean = drop_categories(df_observations_clean, ['urinalysis','microbiology','infectious','immunology_allergy','oncology','ophthalmology_imaging','mental_health','social_determinants','demographics','substance_use','administrative','other'], column='DESCRIPTION', fuzzy=True)


=== Diagnosis for category: urinalysis ===
Variables in category: 28
Unique values in df['DESCRIPTION']: 288

Exact matches found: 27
Examples: ['Glucose [Mass/volume] in Urine by Test strip', 'Bacteria identified in Urine by Culture', 'Ketones [Mass/volume] in Urine by Test strip']

Partial matches found: 33
Examples: ['Glucose [Mass/volume] in Urine by Test strip', 'Glucose', 'Bacteria identified in Urine by Culture']


Los modelos de aprendizaje automático son más precisos cuanto más datos usamos para entrenarlos y este conjunto de datos puede ser demasiado pequeño para conseguir un gran rendimiento. Comprobemos si es así.

En las siguientes subsecciones limpiamos y preparamos los datos porque queremos entrenar el modelo con la mejor información. Los datos pueden estar etiquetados incorrectamente, presentar errores tipográficos, .... Además, es posible que los modelos no acepten valores faltantes (como 'null' o NaN) o ciertos tipos de datos o puede que no convergan si los datos no están escaleados. Finalmente, queremos que los modelos separen la información relevante que las personas no podemos inferir, por lo tanto, podemos usar la información que sí inferimos simplemente (como corregir errores tipográficos ...) para ayudarlos (y ahorrar tiempo y energía).

## 2.3 Limpieza de datos

### Valores faltantes y data types

In [ ]:
df_patients.info()

No se detectan valores faltantes en el conjunto de datos. Eso significa que no están representados por 'NaN' en el conjunto de datos, pero a menudo se usan otros valores para la información faltante como 'desconocido', 'unknown', '?', '9999', ... y vamos estar atentes a esto.

Vamos a reducir este dataset a simplemente las variables AGE convertida a raiz del BIRTHDATE, y GENDER. Posteriormente, vamos a añadir más valores para trabajar sobre un solo dataset.

In [ ]:
# Store in different lists the names of the categorical and numerical columns since they will be treated different 
categorical = df_patients.select_dtypes(include=['object']).columns.tolist()  # for strings 
numerical = df_patients.select_dtypes(include=['int64','float64']).columns.tolist() # for numbers

### Duplicados

In [ ]:
df_patients.duplicated().count() 

El método duplicated() da 'False' si la fila no está duplicada y el método count() solo cuenta los valores 'True', ya que hay 918 valores verdaderos en 918 filas, no hay filas duplicadas (en la página de Kaggle del conjunto de datos se dice que los duplicados ya se eliminaron).

### Rangos y estadística básica

In [ ]:
# Check the stats of numerical features
df_patients.describe(include = np.number).round(2)

### Valores únicos

El método describe() ya mostraba los valores únicos para las variables categóricas. Comprobamos ahora los valores únicos de las características numéricas

# 2.4 Feature importance y analisis de la variable 'target'